In [ ]:
import numpy as np

print("="*80)
print("KALMAN FILTER: HAND CALCULATION EXAMPLE")
print("="*80)
print("\nScenario: Ball moving at constant velocity (simplified 1D case)")
print("  True velocity: 10 pixels/step")
print("  Measurement noise: σ² = 4 (so σ = 2 pixels)")
print("  Process noise: very small (we trust our model)")
print("  Time step: dt = 1.0")
print("\n" + "="*80)

# Initial state: we know nothing!
x_est = np.array([0.0, 0.0])  # [position, velocity]
print("\nINITIAL STATE:")
print(f"  x̂ = {x_est}  (we start with zero estimate)")

# Uncertainty: we're very uncertain at first
P = np.array([[100.0, 0.0],    # High uncertainty in position
              [0.0, 100.0]])     # High uncertainty in velocity
print(f"\n  P (uncertainty) =\n{P}")

# System matrices
F = np.array([[1.0, 1.0],    # pos(k) = pos(k-1) + vel(k-1)*dt
              [0.0, 1.0]])    # vel(k) = vel(k-1)
H = np.array([[1.0, 0.0]])   # We only measure position
Q = np.array([[0.1, 0.0],    # Small process noise
              [0.0, 0.1]])
R = np.array([[4.0]])        # Measurement noise variance = 4

print(f"\n  State transition F =\n{F}")
print(f"  Measurement matrix H = {H}")
print(f"  Process noise Q =\n{Q}")
print(f"  Measurement noise R = {R}")

# True motion (unknown to filter)
true_pos = [0, 10, 20]  # Ball moves 10 pixels per step
true_vel = 10

# Simulated measurements (with noise)
np.random.seed(42)
measurements = [0.0, 11.5, 18.8]  # Noisy measurements

print("\n" + "="*80)

for step in range(3):
    print(f"\n{'█'*80}")
    print(f"ITERATION {step + 1}")
    print(f"{'█'*80}")
    
    print(f"\n📍 GROUND TRUTH (unknown to filter):")
    print(f"   True position: {true_pos[step]:.1f}")
    print(f"   True velocity: {true_vel}")
    
    print(f"\n📏 MEASUREMENT:")
    z = measurements[step]
    print(f"   Measured position: {z:.1f}")
    print(f"   (True: {true_pos[step]:.1f}, Noise: {z - true_pos[step]:.1f})")
    
    # PREDICT STEP
    print(f"\n{'─'*80}")
    print("STEP 1: PREDICT")
    print(f"{'─'*80}")
    
    print(f"\n  Current estimate before prediction:")
    print(f"    x̂ = {x_est}")
    
    # State prediction
    x_pred = F @ x_est
    print(f"\n  State prediction: x̂⁻ = F × x̂")
    print(f"    F × x̂ = {F} × {x_est}")
    print(f"    Calculation: pos⁻ = {x_est[0]:.1f} + {x_est[1]:.1f}×1 = {x_pred[0]:.1f}")
    print(f"                 vel⁻ = {x_est[1]:.1f}")
    print(f"    x̂⁻ = {x_pred}")
    
    # Covariance prediction
    P_pred = F @ P @ F.T + Q
    print(f"\n  Uncertainty prediction: P⁻ = F×P×F^T + Q")
    print(f"    P⁻ =\n{P_pred}")
    print(f"    (Uncertainty grows because model isn't perfect)")
    
    # UPDATE STEP
    print(f"\n{'─'*80}")
    print("STEP 2: UPDATE")
    print(f"{'─'*80}")
    
    # Innovation
    y_innov = z - (H @ x_pred)[0]
    print(f"\n  Innovation (surprise): y = z - H×x̂⁻")
    print(f"    y = {z:.1f} - {(H @ x_pred)[0]:.1f} = {y_innov:.1f}")
    print(f"    (How different is measurement from prediction)")
    
    # Innovation covariance
    S = (H @ P_pred @ H.T + R)[0, 0]
    print(f"\n  Innovation uncertainty: S = H×P⁻×H^T + R")
    print(f"    S = {S:.2f}")
    
    # Kalman gain
    K = (P_pred @ H.T) / S
    print(f"\n  Kalman Gain: K = P⁻×H^T / S")
    print(f"    K = {K.flatten()}")
    print(f"    Interpretation:")
    print(f"      K[0] = {K[0, 0]:.3f} → Update position with {K[0, 0]*100:.1f}% of innovation")
    print(f"      K[1] = {K[1, 0]:.3f} → Update velocity with {K[1, 0]*100:.1f}% of innovation")
    
    # State update
    x_est = x_pred + (K * y_innov).flatten()
    print(f"\n  State update: x̂ = x̂⁻ + K×y")
    print(f"    x̂⁻ = {x_pred}")
    print(f"    K×y = {K.flatten()} × {y_innov:.1f} = {(K * y_innov).flatten()}")
    print(f"    x̂ = {x_pred} + {(K * y_innov).flatten()}")
    print(f"    x̂ = {x_est}")
    
    # Covariance update
    I = np.eye(2)
    P = (I - K @ H) @ P_pred
    print(f"\n  Uncertainty update: P = (I - K×H)×P⁻")
    print(f"    P =\n{P}")
    print(f"    (Uncertainty decreases because we got new info!)")
    
    # Results
    print(f"\n{'─'*80}")
    print("FINAL ESTIMATES:")
    print(f"{'─'*80}")
    print(f"  Estimated position: {x_est[0]:.2f}  (True: {true_pos[step]:.1f}, Error: {abs(x_est[0] - true_pos[step]):.2f})")
    print(f"  Estimated velocity: {x_est[1]:.2f}  (True: {true_vel:.1f}, Error: {abs(x_est[1] - true_vel):.2f})")
    
    if step < 2:
        print(f"\n  ⏭️  Moving to next iteration...")
        print(f"     (Current estimate becomes input for next predict step)")

print("\n" + "="*80)
print("KEY OBSERVATIONS:")
print("="*80)
print("1. Filter started knowing NOTHING (0 position, 0 velocity)")
print("2. By iteration 3, it estimated velocity ≈ 10 from position measurements alone!")
print("3. Kalman gain automatically adjusted trust between prediction and measurement")
print("4. Uncertainty (P) decreased each time we got a measurement")
print("5. The filter is learning both position AND velocity from position-only measurements")
print("="*80)

# ⚽ Kalman Filter Ball Chase

The red ball uses a Kalman filter to predict where the blue ball will be. The blue ball moves in a circular path while the Kalman filter learns its trajectory and guides the red ball to intercept it.

Adjust the measurement noise to see how the filter adapts to sensor uncertainty!

## Interactive HTML5 Canvas Animation

Run this cell to see the animation with full JavaScript controls!

In [ ]:
from IPython.display import HTML

html_code = '''
<!DOCTYPE html>
<html>
<head>
    <style>
        body {
            background-color: #1a1a2e;
            margin: 0;
            padding: 20px;
            font-family: Arial, sans-serif;
        }
        #canvas-container {
            text-align: center;
        }
        canvas {
            background-color: #1a1a2e;
            border: 2px solid #667eea;
            border-radius: 10px;
            cursor: pointer;
        }
        .controls {
            margin: 20px auto;
            max-width: 700px;
            background: #2a2a3e;
            padding: 20px;
            border-radius: 10px;
        }
        button {
            background: #667eea;
            color: white;
            border: none;
            padding: 10px 20px;
            margin: 5px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 14px;
        }
        button:hover { background: #5568d3; }
        .slider-container {
            margin: 15px 0;
            color: white;
        }
        input[type="range"] {
            width: 100%;
            margin: 10px 0;
        }
        label {
            display: block;
            margin-bottom: 5px;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div id="canvas-container">
        <canvas id="canvas" width="700" height="700"></canvas>
    </div>
    <div class="controls">
        <div>
            <button onclick="resetSimulation()">🔄 Reset</button>
            <button onclick="togglePause()">⏸️ Pause</button>
        </div>
        <div class="slider-container">
            <label>Measurement Noise (σ²): <span id="noise-value">40</span></label>
            <input type="range" id="noise-slider" min="1" max="150" value="40" 
                   oninput="updateNoise(this.value)">
        </div>
        <div class="slider-container">
            <label>Circle Radius: <span id="radius-value">120</span> px</label>
            <input type="range" id="radius-slider" min="50" max="200" step="10" value="120" 
                   oninput="updateRadius(this.value)">
        </div>
    </div>

    <script>
        const canvas = document.getElementById('canvas');
        const ctx = canvas.getContext('2d');
        
        let paused = false;
        
        // Kalman Filter Class
        class KalmanFilter2D {
            constructor(processNoise = 0.1, measurementNoise = 40) {
                this.state = [350, 350, 0, 0];
                this.P = [
                    [1000, 0, 0, 0],
                    [0, 1000, 0, 0],
                    [0, 0, 100, 0],
                    [0, 0, 0, 100]
                ];
                this.Q = [
                    [processNoise, 0, 0, 0],
                    [0, processNoise, 0, 0],
                    [0, 0, processNoise * 10, 0],
                    [0, 0, 0, processNoise * 10]
                ];
                this.R = [
                    [measurementNoise, 0],
                    [0, measurementNoise]
                ];
                this.dt = 0.016;
                this.F = [
                    [1, 0, this.dt, 0],
                    [0, 1, 0, this.dt],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]
                ];
                this.H = [
                    [1, 0, 0, 0],
                    [0, 1, 0, 0]
                ];
            }
            
            predict() {
                this.state = this.matMulVec(this.F, this.state);
                this.P = this.matAdd(this.matMul(this.matMul(this.F, this.P), this.transpose(this.F)), this.Q);
            }
            
            update(measurement) {
                const innovation = [measurement[0] - this.state[0], measurement[1] - this.state[1]];
                const S = this.matAdd(this.matMul(this.matMul(this.H, this.P), this.transpose(this.H)), this.R);
                const K = this.matMul(this.matMul(this.P, this.transpose(this.H)), this.inverse2x2(S));
                const correction = this.matMulVec(K, innovation);
                this.state = this.state.map((s, i) => s + correction[i]);
                const I = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]];
                this.P = this.matMul(this.matSub(I, this.matMul(K, this.H)), this.P);
            }
            
            setMeasurementNoise(noise) {
                this.R = [[noise, 0], [0, noise]];
            }
            
            matMul(A, B) {
                const result = [];
                for(let i = 0; i < A.length; i++) {
                    result[i] = [];
                    for(let j = 0; j < B[0].length; j++) {
                        result[i][j] = 0;
                        for(let k = 0; k < B.length; k++)
                            result[i][j] += A[i][k] * B[k][j];
                    }
                }
                return result;
            }
            
            matMulVec(A, v) {
                return A.map(row => row.reduce((sum, val, i) => sum + val * v[i], 0));
            }
            
            matAdd(A, B) {
                return A.map((row, i) => row.map((val, j) => val + B[i][j]));
            }
            
            matSub(A, B) {
                return A.map((row, i) => row.map((val, j) => val - B[i][j]));
            }
            
            transpose(A) {
                return A[0].map((_, i) => A.map(row => row[i]));
            }
            
            inverse2x2(m) {
                const det = m[0][0] * m[1][1] - m[0][1] * m[1][0];
                return [[m[1][1]/det, -m[0][1]/det], [-m[1][0]/det, m[0][0]/det]];
            }
        }
        
        let sim = {
            center: {x: 350, y: 350},
            radius: 120,
            blueAngle: 0,
            blueSpeed: 0.008,
            blueRadius: 12,
            redPos: {x: 350 - 120, y: 350},
            redRadius: 12,
            blueTrail: [],
            redTrail: [],
            predTrail: [],
            maxTrail: 200,
            caught: false,
            catchDistance: 25,
            measurementNoise: 40,
            kalman: null
        };
        
        sim.redSpeed = sim.radius * sim.blueSpeed;
        sim.kalman = new KalmanFilter2D(0.1, sim.measurementNoise);
        
        function resetSimulation() {
            sim.blueAngle = 0;
            sim.redPos = {x: sim.center.x - sim.radius, y: sim.center.y};
            sim.blueTrail = [];
            sim.redTrail = [];
            sim.predTrail = [];
            sim.caught = false;
            sim.redSpeed = sim.radius * sim.blueSpeed;
            sim.kalman = new KalmanFilter2D(0.1, sim.measurementNoise);
        }
        
        function togglePause() {
            paused = !paused;
            document.querySelector('button[onclick="togglePause()"]').textContent = 
                paused ? '▶️ Resume' : '⏸️ Pause';
        }
        
        function updateNoise(value) {
            sim.measurementNoise = parseFloat(value);
            sim.kalman.setMeasurementNoise(sim.measurementNoise);
            document.getElementById('noise-value').textContent = value;
        }
        
        function updateRadius(value) {
            sim.radius = parseFloat(value);
            document.getElementById('radius-value').textContent = value;
            resetSimulation();
        }
        
        function update() {
            if (paused) return;
            
            sim.blueAngle += sim.blueSpeed;
            const bluePos = {
                x: sim.center.x + sim.radius * Math.cos(sim.blueAngle),
                y: sim.center.y + sim.radius * Math.sin(sim.blueAngle)
            };
            
            const noise = {
                x: (Math.random() - 0.5) * 2 * Math.sqrt(sim.measurementNoise),
                y: (Math.random() - 0.5) * 2 * Math.sqrt(sim.measurementNoise)
            };
            const measuredPos = [bluePos.x + noise.x, bluePos.y + noise.y];
            
            sim.kalman.predict();
            sim.kalman.update(measuredPos);
            
            const futureSteps = 5;
            const futureAngle = sim.blueAngle + sim.blueSpeed * futureSteps;
            const predPos = {
                x: sim.center.x + sim.radius * Math.cos(futureAngle),
                y: sim.center.y + sim.radius * Math.sin(futureAngle)
            };
            
            const dx = predPos.x - sim.redPos.x;
            const dy = predPos.y - sim.redPos.y;
            const dist = Math.sqrt(dx*dx + dy*dy);
            
            if (dist > 0) {
                sim.redPos.x += (dx/dist) * sim.redSpeed;
                sim.redPos.y += (dy/dist) * sim.redSpeed;
            }
            
            const catchDist = Math.sqrt(
                Math.pow(bluePos.x - sim.redPos.x, 2) + 
                Math.pow(bluePos.y - sim.redPos.y, 2)
            );
            sim.caught = catchDist < sim.catchDistance;
            
            sim.blueTrail.push({...bluePos});
            sim.redTrail.push({...sim.redPos});
            sim.predTrail.push({...predPos});
            
            if (sim.blueTrail.length > sim.maxTrail) {
                sim.blueTrail.shift();
                sim.redTrail.shift();
                sim.predTrail.shift();
            }
            
            return {bluePos, predPos, catchDist};
        }
        
        function draw() {
            const {bluePos, predPos, catchDist} = update() || {};
            
            ctx.fillStyle = '#1a1a2e';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
            
            ctx.strokeStyle = 'rgba(255, 255, 255, 0.3)';
            ctx.lineWidth = 2;
            ctx.setLineDash([5, 5]);
            ctx.beginPath();
            ctx.arc(sim.center.x, sim.center.y, sim.radius, 0, Math.PI * 2);
            ctx.stroke();
            ctx.setLineDash([]);
            
            if (sim.blueTrail.length > 1) {
                ctx.strokeStyle = 'rgba(52, 152, 219, 0.3)';
                ctx.lineWidth = 1.5;
                ctx.beginPath();
                ctx.moveTo(sim.blueTrail[0].x, sim.blueTrail[0].y);
                for(let i = 1; i < sim.blueTrail.length; i++) {
                    ctx.lineTo(sim.blueTrail[i].x, sim.blueTrail[i].y);
                }
                ctx.stroke();
                
                ctx.strokeStyle = 'rgba(231, 76, 60, 0.3)';
                ctx.beginPath();
                ctx.moveTo(sim.redTrail[0].x, sim.redTrail[0].y);
                for(let i = 1; i < sim.redTrail.length; i++) {
                    ctx.lineTo(sim.redTrail[i].x, sim.redTrail[i].y);
                }
                ctx.stroke();
            }
            
            if (predPos) {
                ctx.strokeStyle = 'rgba(46, 204, 113, 0.5)';
                ctx.lineWidth = 1.5;
                ctx.setLineDash([5, 3]);
                ctx.beginPath();
                ctx.moveTo(sim.redPos.x, sim.redPos.y);
                ctx.lineTo(predPos.x, predPos.y);
                ctx.stroke();
                ctx.setLineDash([]);
                
                ctx.fillStyle = 'rgba(46, 204, 113, 0.7)';
                ctx.beginPath();
                ctx.arc(predPos.x, predPos.y, 6, 0, Math.PI * 2);
                ctx.fill();
            }
            
            if (bluePos) {
                ctx.fillStyle = '#3498db';
                ctx.beginPath();
                ctx.arc(bluePos.x, bluePos.y, sim.blueRadius, 0, Math.PI * 2);
                ctx.fill();
            }
            
            ctx.fillStyle = '#e74c3c';
            ctx.beginPath();
            ctx.arc(sim.redPos.x, sim.redPos.y, sim.redRadius, 0, Math.PI * 2);
            ctx.fill();
            
            const statusText = sim.caught ? '🎉 Caught! ✨' : '🎯 Chasing...';
            const statusColor = sim.caught ? '#ff9800' : '#667eea';
            
            ctx.fillStyle = statusColor;
            ctx.fillRect(260, 630, 180, 40);
            ctx.fillStyle = 'white';
            ctx.font = 'bold 16px Arial';
            ctx.textAlign = 'center';
            ctx.fillText(statusText, 350, 655);
            
            if (catchDist !== undefined) {
                ctx.fillStyle = 'white';
                ctx.font = '10px monospace';
                ctx.fillText(`Distance: ${catchDist.toFixed(1)} px`, 350, 30);
            }
            
            requestAnimationFrame(draw);
        }
        
        draw();
    </script>
</body>
</html>
'''

HTML(html_code)

---

# 📚 Tutorial: Understanding the Kalman Filter

Now let's dive deep into how the Kalman filter actually works with Python implementations and detailed mathematical examples.

## Part 1: Python Implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

class KalmanFilter2D:
    """
    2D Kalman Filter for tracking position and velocity.
    
    State vector: [x, y, vx, vy]
    - x, y: Position in 2D space
    - vx, vy: Velocity in x and y directions
    
    Measurement vector: [x_measured, y_measured]
    - We only measure position, not velocity
    """
    
    def __init__(self, process_noise=0.1, measurement_noise=40, dt=0.016):
        """
        Initialize Kalman Filter.
        
        Parameters:
        -----------
        process_noise : float
            How much we trust our motion model (lower = more trust)
        measurement_noise : float  
            How much noise is in our sensor readings (lower = cleaner sensor)
        dt : float
            Time step between measurements (seconds)
        """
        self.dt = dt
        
        # Initial state: [x, y, vx, vy]
        self.state = np.array([0.0, 0.0, 0.0, 0.0])
        
        # State covariance matrix (4x4) - uncertainty in our estimate
        self.P = np.array([
            [1000, 0, 0, 0],      # High initial uncertainty in x
            [0, 1000, 0, 0],      # High initial uncertainty in y
            [0, 0, 100, 0],       # Medium uncertainty in vx
            [0, 0, 0, 100]        # Medium uncertainty in vy
        ], dtype=float)
        
        # Process noise covariance (4x4) - uncertainty in motion model
        self.Q = np.array([
            [process_noise, 0, 0, 0],
            [0, process_noise, 0, 0],
            [0, 0, process_noise * 10, 0],  # Velocity is less predictable
            [0, 0, 0, process_noise * 10]
        ], dtype=float)
        
        # Measurement noise covariance (2x2) - sensor noise
        self.R = np.array([
            [measurement_noise, 0],
            [0, measurement_noise]
        ], dtype=float)
        
        # State transition matrix F (4x4) - constant velocity model
        # New position = old position + velocity * dt
        # New velocity = old velocity (constant)
        self.F = np.array([
            [1, 0, dt, 0],   # x(k) = x(k-1) + vx*dt
            [0, 1, 0, dt],   # y(k) = y(k-1) + vy*dt  
            [0, 0, 1, 0],    # vx(k) = vx(k-1)
            [0, 0, 0, 1]     # vy(k) = vy(k-1)
        ], dtype=float)
        
        # Measurement matrix H (2x4) - we only observe position
        self.H = np.array([
            [1, 0, 0, 0],    # Measure x
            [0, 1, 0, 0]     # Measure y
        ], dtype=float)
    
    def predict(self):
        """
        PREDICTION STEP: Project state forward in time using motion model.
        
        This is where we use physics to predict where the object will be
        before we get a new measurement.
        """
        # State prediction: x̂⁻(k) = F · x̂(k-1)
        self.state = self.F @ self.state
        
        # Covariance prediction: P⁻(k) = F·P(k-1)·Fᵀ + Q
        # Uncertainty grows because motion model isn't perfect
        self.P = self.F @ self.P @ self.F.T + self.Q
        
        return self.state.copy()
    
    def update(self, measurement):
        """
        UPDATE STEP: Correct prediction using new measurement.
        
        Parameters:
        -----------
        measurement : array-like, shape (2,)
            Measured [x, y] position (noisy)
        
        This is where we blend our prediction with the sensor reading
        to get the best estimate.
        """
        measurement = np.array(measurement)
        
        # Innovation (residual): y(k) = z(k) - H·x̂⁻(k)
        # How different is the measurement from what we predicted?
        innovation = measurement - self.H @ self.state
        
        # Innovation covariance: S(k) = H·P⁻(k)·Hᵀ + R
        # How uncertain is this innovation?
        S = self.H @ self.P @ self.H.T + self.R
        
        # Kalman gain: K(k) = P⁻(k)·Hᵀ·S⁻¹(k)
        # THE MAGIC: How much should we trust the measurement vs prediction?
        K = self.P @ self.H.T @ np.linalg.inv(S)
        
        # State update: x̂(k) = x̂⁻(k) + K(k)·y(k)
        # Blend prediction and measurement using Kalman gain
        self.state = self.state + K @ innovation
        
        # Covariance update: P(k) = (I - K(k)·H)·P⁻(k)
        # Uncertainty decreases because we got new information
        I = np.eye(4)
        self.P = (I - K @ self.H) @ self.P
        
        return self.state.copy(), K
    
    def get_position(self):
        """Return current position estimate."""
        return self.state[0], self.state[1]
    
    def get_velocity(self):
        """Return current velocity estimate."""
        return self.state[2], self.state[3]

print("✅ KalmanFilter2D class defined!")

## Part 2: Step-by-Step Example with Real Numbers

Let's track a ball moving in a circle and see exactly how the Kalman filter processes each measurement.

In [ ]:
# Create a Kalman filter
kf = KalmanFilter2D(process_noise=0.1, measurement_noise=40, dt=0.016)

print("="*70)
print("KALMAN FILTER ITERATION EXAMPLE")
print("="*70)

# Simulate a ball moving in a circle
center = np.array([100.0, 100.0])
radius = 50.0
angle = 0.0
angular_velocity = 0.1  # radians per time step

print("\n📍 Scenario: Ball moving in a circle")
print(f"   Center: {center}")
print(f"   Radius: {radius}")
print(f"   Angular velocity: {angular_velocity} rad/step")

# Run 3 iterations to show the process
for iteration in range(3):
    print(f"\n{'='*70}")
    print(f"ITERATION {iteration + 1}")
    print(f"{'='*70}")
    
    # True position (what's really happening)
    true_pos = center + radius * np.array([np.cos(angle), np.sin(angle)])
    true_vel = radius * angular_velocity * np.array([-np.sin(angle), np.cos(angle)])
    
    print(f"\n1️⃣  TRUE STATE (unknown to filter):")
    print(f"   Position: [{true_pos[0]:.2f}, {true_pos[1]:.2f}]")
    print(f"   Velocity: [{true_vel[0]:.2f}, {true_vel[1]:.2f}]")
    
    # Add measurement noise
    noise = np.random.randn(2) * np.sqrt(40)
    measured_pos = true_pos + noise
    
    print(f"\n2️⃣  NOISY MEASUREMENT (what sensor sees):")
    print(f"   Measured: [{measured_pos[0]:.2f}, {measured_pos[1]:.2f}]")
    print(f"   Noise added: [{noise[0]:.2f}, {noise[1]:.2f}]")
    print(f"   Error: {np.linalg.norm(measured_pos - true_pos):.2f} pixels")
    
    # PREDICT STEP
    print(f"\n3️⃣  PREDICT STEP:")
    print(f"   Before: x̂ = [{kf.state[0]:.2f}, {kf.state[1]:.2f}, {kf.state[2]:.2f}, {kf.state[3]:.2f}]")
    predicted_state = kf.predict()
    print(f"   After:  x̂⁻= [{predicted_state[0]:.2f}, {predicted_state[1]:.2f}, {predicted_state[2]:.2f}, {predicted_state[3]:.2f}]")
    print(f"   (Used motion model: new_pos = old_pos + velocity*dt)")
    
    # UPDATE STEP
    print(f"\n4️⃣  UPDATE STEP:")
    updated_state, kalman_gain = kf.update(measured_pos)
    print(f"   Innovation: [{measured_pos[0] - predicted_state[0]:.2f}, {measured_pos[1] - predicted_state[1]:.2f}]")
    print(f"   (How different is measurement from prediction)")
    print(f"\n   Kalman Gain K (4x2 matrix):")
    print(f"   {kalman_gain}")
    print(f"   (Controls how much we trust measurement vs prediction)")
    print(f"\n   Final estimate: x̂ = [{updated_state[0]:.2f}, {updated_state[1]:.2f}, {updated_state[2]:.2f}, {updated_state[3]:.2f}]")
    
    # Compare with truth
    pos_error = np.linalg.norm(updated_state[:2] - true_pos)
    vel_error = np.linalg.norm(updated_state[2:] - true_vel)
    
    print(f"\n5️⃣  RESULTS:")
    print(f"   Position error: {pos_error:.2f} pixels")
    print(f"   Velocity error: {vel_error:.2f} pixels/sec")
    print(f"   ✨ Filter smoothed out {np.linalg.norm(noise) - pos_error:.2f} pixels of noise!")
    
    # Move to next time step
    angle += angular_velocity

print(f"\n{'='*70}")
print("Key Insights:")
print("1. Filter starts with poor estimates but improves over time")
print("2. Kalman gain automatically balances prediction vs measurement")
print("3. Filter estimates velocity even though we only measure position!")
print("4. Final estimates are much better than raw noisy measurements")
print(f"{'='*70}")

## 🔢 Numeric Example: Hand Calculations

Let's work through 3 iterations **by hand** with simple numbers to see exactly what the Kalman filter does at each step.

**Scenario:** Track a ball moving to the right at constant velocity.
- True position starts at x=0, moving right at 10 pixels/step
- We measure position (with noise) but want to estimate both position AND velocity
- Time step dt = 1.0 (simplified)

## Part 3: Visual Comparison - With vs Without Kalman Filter

In [ ]:
# Run a longer simulation and plot results with catch detection
np.random.seed(42)

# Setup
n_steps = 100
center = np.array([0.0, 0.0])
radius = 50.0
angles = np.linspace(0, 4*np.pi, n_steps)  # Two full circles

# Red ball chase parameters
red_speed = 0.96  # pixels per frame
catch_distance = 25  # pixels

# Storage
true_positions = []
measured_positions = []
kalman_positions = []
red_positions = []
distances = []
caught = False
catch_time = None

# Initial red position
red_pos = np.array([-radius, 0.0])

# Create new filter
kf = KalmanFilter2D(process_noise=0.1, measurement_noise=40, dt=0.016)

# Simulate
for i, angle in enumerate(angles):
    # True position of blue ball
    true_pos = center + radius * np.array([np.cos(angle), np.sin(angle)])
    true_positions.append(true_pos)
    
    # Noisy measurement
    noise = np.random.randn(2) * np.sqrt(40)
    measured_pos = true_pos + noise
    measured_positions.append(measured_pos)
    
    # Kalman filter
    kf.predict()
    kf.update(measured_pos)
    kalman_pos = np.array(kf.get_position())
    kalman_positions.append(kalman_pos)
    
    # Predict future position (5 steps ahead)
    future_angle = angle + 0.008 * 5 * (4*np.pi / n_steps)
    predicted_pos = center + radius * np.array([np.cos(future_angle), np.sin(future_angle)])
    
    # Move red ball toward predicted position
    direction = predicted_pos - red_pos
    dist_to_target = np.linalg.norm(direction)
    if dist_to_target > 0:
        red_pos += (direction / dist_to_target) * red_speed
    
    red_positions.append(red_pos.copy())
    
    # Check if caught
    distance_to_blue = np.linalg.norm(true_pos - red_pos)
    distances.append(distance_to_blue)
    
    if not caught and distance_to_blue < catch_distance:
        caught = True
        catch_time = i

# Convert to arrays
true_positions = np.array(true_positions)
measured_positions = np.array(measured_positions)
kalman_positions = np.array(kalman_positions)
red_positions = np.array(red_positions)
distances = np.array(distances)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Left plot: Trajectories with catch detection
ax = axes[0]
ax.set_facecolor('#1a1a2e')
ax.plot(true_positions[:, 0], true_positions[:, 1], 
        'g-', linewidth=3, label='True Path (Blue Ball)', alpha=0.8)
ax.scatter(measured_positions[:, 0], measured_positions[:, 1], 
          c='#3498db', s=20, label='Noisy Measurements', alpha=0.5)
ax.plot(kalman_positions[:, 0], kalman_positions[:, 1], 
        'r-', linewidth=2, label='Kalman Estimate', alpha=0.9)
ax.plot(red_positions[:, 0], red_positions[:, 1], 
        color='#e74c3c', linewidth=2.5, label='Red Ball (Chaser)', alpha=0.9, linestyle='--')

# Mark catch point if caught
if caught:
    catch_pos = true_positions[catch_time]
    ax.scatter([catch_pos[0]], [catch_pos[1]], 
              s=500, c='#ff9800', marker='*', 
              edgecolors='white', linewidths=2,
              label=f'Caught! (t={catch_time})', zorder=5)

ax.set_xlabel('X Position', color='white', fontsize=12)
ax.set_ylabel('Y Position', color='white', fontsize=12)
title_text = 'Kalman Filter Ball Chase'
if caught:
    title_text += ' - ✨ CAUGHT! ✨'
ax.set_title(title_text, color='white', fontsize=14, weight='bold')
ax.legend(facecolor='#2a2a3e', edgecolor='white', fontsize=11)
ax.grid(True, alpha=0.2, color='white')
ax.tick_params(colors='white')
ax.spines['bottom'].set_color('white')
ax.spines['left'].set_color('white')
ax.spines['top'].set_color('white')
ax.spines['right'].set_color('white')
ax.set_aspect('equal')

# Right plot: Distance over time with catch threshold
ax = axes[1]
ax.set_facecolor('#1a1a2e')

ax.plot(distances, color='#9b59b6', linewidth=2.5, label='Distance to Blue Ball')
ax.axhline(catch_distance, color='#ff9800', linestyle='--', linewidth=2, 
          label=f'Catch Threshold ({catch_distance} px)', alpha=0.7)

# Highlight caught region
if caught:
    ax.axvspan(catch_time, n_steps, alpha=0.2, color='#ff9800', label='Caught!')
    ax.scatter([catch_time], [distances[catch_time]], 
              s=200, c='#ff9800', marker='*', 
              edgecolors='white', linewidths=2, zorder=5)

ax.set_xlabel('Time Step', color='white', fontsize=12)
ax.set_ylabel('Distance (pixels)', color='white', fontsize=12)
ax.set_title('Red Ball Distance to Blue Ball Over Time', color='white', fontsize=14, weight='bold')
ax.legend(facecolor='#2a2a3e', edgecolor='white', fontsize=10)
ax.grid(True, alpha=0.2, color='white')
ax.tick_params(colors='white')
ax.spines['bottom'].set_color('white')
ax.spines['left'].set_color('white')
ax.spines['top'].set_color('white')
ax.spines['right'].set_color('white')

fig.patch.set_facecolor('#1a1a2e')
plt.tight_layout()
plt.show()

# Performance summary
measured_errors = np.linalg.norm(measured_positions - true_positions, axis=1)
kalman_errors = np.linalg.norm(kalman_positions - true_positions, axis=1)
improvement = (measured_errors.mean() - kalman_errors.mean()) / measured_errors.mean() * 100

print(f"\n📊 Performance Summary:")
print(f"   Average measurement error: {measured_errors.mean():.2f} pixels")
print(f"   Average Kalman filter error: {kalman_errors.mean():.2f} pixels")
print(f"   Improvement: {improvement:.1f}% reduction in error!")
print(f"\n🎯 Chase Performance:")
print(f"   Minimum distance achieved: {distances.min():.2f} pixels")
if caught:
    print(f"   ✨ Blue ball CAUGHT at time step {catch_time} (distance: {distances[catch_time]:.2f} px)")
else:
    print(f"   ❌ Blue ball NOT caught (closest: {distances.min():.2f} px, needed: <{catch_distance} px)")

## Part 4: Mathematical Deep Dive

### The Kalman Filter Equations Explained

The Kalman filter has two main phases that repeat:

#### **PREDICTION PHASE** (Time Update)

We use our motion model to predict where the object will be:

$$\hat{\mathbf{x}}^-(k) = \mathbf{F} \cdot \hat{\mathbf{x}}(k-1)$$

Where:
- $\hat{\mathbf{x}}^-(k)$ = Predicted state at time $k$ (before measurement)
- $\mathbf{F}$ = State transition matrix (4×4)
- $\hat{\mathbf{x}}(k-1)$ = Previous best estimate

For our constant velocity model:

$$\mathbf{F} = \begin{bmatrix} 
1 & 0 & \Delta t & 0 \\
0 & 1 & 0 & \Delta t \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}$$

This means:
- New $x$ = Old $x$ + $v_x \cdot \Delta t$
- New $y$ = Old $y$ + $v_y \cdot \Delta t$  
- Velocity stays constant

We also predict how uncertain we are:

$$\mathbf{P}^-(k) = \mathbf{F} \cdot \mathbf{P}(k-1) \cdot \mathbf{F}^T + \mathbf{Q}$$

Where:
- $\mathbf{P}^-(k)$ = Predicted covariance (uncertainty)
- $\mathbf{Q}$ = Process noise (4×4) - How much error our model has

---

#### **UPDATE PHASE** (Measurement Update)

We get a new measurement and use it to correct our prediction:

**Step 1: Calculate innovation (residual)**

$$\mathbf{y}(k) = \mathbf{z}(k) - \mathbf{H} \cdot \hat{\mathbf{x}}^-(k)$$

Where:
- $\mathbf{y}(k)$ = Innovation (how surprised we are)
- $\mathbf{z}(k)$ = Measurement (2×1: observed x,y)
- $\mathbf{H}$ = Measurement matrix (2×4)

$$\mathbf{H} = \begin{bmatrix} 
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0
\end{bmatrix}$$

This extracts just position from [x, y, vx, vy]

**Step 2: Calculate innovation covariance**

$$\mathbf{S}(k) = \mathbf{H} \cdot \mathbf{P}^-(k) \cdot \mathbf{H}^T + \mathbf{R}$$

Where:
- $\mathbf{S}(k)$ = Innovation covariance (2×2)
- $\mathbf{R}$ = Measurement noise covariance (2×2)

**Step 3: Calculate Kalman Gain (THE MAGIC!)**

$$\mathbf{K}(k) = \mathbf{P}^-(k) \cdot \mathbf{H}^T \cdot \mathbf{S}^{-1}(k)$$

The Kalman gain $\mathbf{K}$ (4×2 matrix) automatically decides:
- If $\mathbf{R}$ is large (noisy sensor) → $\mathbf{K}$ is small → Trust prediction more
- If $\mathbf{P}$ is large (uncertain prediction) → $\mathbf{K}$ is large → Trust measurement more

**Step 4: Update state estimate**

$$\hat{\mathbf{x}}(k) = \hat{\mathbf{x}}^-(k) + \mathbf{K}(k) \cdot \mathbf{y}(k)$$

Blend prediction with measurement weighted by Kalman gain

**Step 5: Update covariance**

$$\mathbf{P}(k) = (\mathbf{I} - \mathbf{K}(k) \cdot \mathbf{H}) \cdot \mathbf{P}^-(k)$$

Uncertainty decreases because we got new information!

---

### Why It's Optimal

Under these assumptions:
1. Linear system dynamics
2. Gaussian noise (process and measurement)
3. Known noise statistics

The Kalman filter **provably minimizes** the mean squared error! No other linear estimator can do better.

### Matrix Dimensions Summary

| Matrix | Dimensions | Meaning |
|--------|-----------|----------|
| $\mathbf{x}$ | 4×1 | State vector [x, y, vx, vy] |
| $\mathbf{z}$ | 2×1 | Measurement [x, y] |
| $\mathbf{F}$ | 4×4 | State transition (motion model) |
| $\mathbf{H}$ | 2×4 | Measurement model |
| $\mathbf{P}$ | 4×4 | State covariance (uncertainty) |
| $\mathbf{Q}$ | 4×4 | Process noise covariance |
| $\mathbf{R}$ | 2×2 | Measurement noise covariance |
| $\mathbf{K}$ | 4×2 | Kalman gain (blend weight) |
| $\mathbf{S}$ | 2×2 | Innovation covariance |

---

## 🎯 Key Takeaways

1. **The Kalman filter is a predictor-corrector algorithm**
   - Predict where object will be using physics
   - Correct prediction using noisy measurement

2. **It automatically balances trust**
   - Kalman gain adjusts based on uncertainty
   - No manual tuning needed (if noise stats are known)

3. **It estimates hidden states**
   - Measure position → Estimate velocity too!
   - Infers what you can't directly observe

4. **It's optimal (under assumptions)**
   - Minimizes mean squared error
   - Best possible linear estimator

5. **Real-world uses**
   - GPS navigation
   - Spacecraft tracking  
   - Self-driving cars
   - Stock market prediction
   - Weather forecasting